# Part 3: Advanced Analysis

In this part, we will implement advanced analysis techniques for physiological time series data, including time-domain feature extraction, frequency analysis, and wavelet transforms.

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
import pywt

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('notebook')

## 1. Time-Domain Feature Extraction

Implement the `extract_time_domain_features` function to extract various time-domain features from physiological signals.

In [9]:
def extract_time_domain_features(data, window_size=60):
    """Extract time-domain features from physiological signals.
    
    Parameters:
    -----------
    data : pandas.DataFrame
    
        Input data with columns: ['timestamp', 'heart_rate', 'eda', 'temperature', 'subject_id', 'session']
    window_size : int, optional
        Size of the rolling window in seconds, default=60
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame containing extracted features for each window
    """
    # Convert window_size from seconds to number of samples
    # Assuming data is sampled at 1 Hz (1 sample per second)

    # I sampled my data at 4 Hz, so there are 240 samples in a window of 60
    window_samples = 4 * window_size
    
    # Initialize DataFrame for features
    features = pd.DataFrame()
    features['timestamp'] = data['timestamp']
    features['subject_id'] = data['subject_id']
    features['session'] = data['session']

    # seems like heart_rate is done for us already... Is this not what we did in part 2??
    
    # process each physiological signal
    for signal_name in ['heart_rate', 'eda', 'temperature']:
        if signal_name in data.columns:
            # basic statistics using rolling window
            features[f'{signal_name}_mean'] = data[signal_name].rolling(window=window_samples).mean()
            features[f'{signal_name}_std'] = data[signal_name].rolling(window=window_samples).std()
            features[f'{signal_name}_min'] = data[signal_name].rolling(window=window_samples).min()
            features[f'{signal_name}_max'] = data[signal_name].rolling(window=window_samples).max()
    
    # heart rate variability measures
    if 'heart_rate' in data.columns:
        # convert HR to RR intervals in milliseconds (ms)
        rr_intervals = 60000 / data['heart_rate']
        # calculate successive differences between RR intervals
        rr_diff = rr_intervals.diff()
        # RMSSD (Root Mean Square of Successive Differences) in ms
        features['hrv_rmssd_ms'] = np.sqrt(
            rr_diff.abs().rolling(window=window_samples).apply(
                lambda x: np.mean(x**2)
            )
        )
        
        # SDNN (Standard Deviation of NN intervals) in ms
        features['hrv_sdnn_ms'] = rr_intervals.rolling(window=window_samples).std()
        # pNN50 (Percentage of successive RR intervals differing by >50ms)
        features['hrv_pnn50_percent'] = rr_diff.rolling(window=window_samples).apply(
            lambda x: 100 * np.sum(np.abs(x) > 50) / len(x) if len(x) > 0 else 0
        )
    
    features = features.dropna()
    
    return features

## 2. Frequency Analysis

Implement the `analyze_frequency_components` function to perform frequency-domain analysis on the signals.

In [10]:
def analyze_frequency_components(data, sampling_rate, window_size=60):
    """Perform frequency-domain analysis on physiological signals.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input data with columns: ['timestamp', 'heart_rate', 'eda', 'temperature', 'subject_id', 'session']
    sampling_rate : float
        Sampling rate of the signal in Hz
    window_size : int, optional
        Size of the analysis window in seconds, default=60
        
    Returns:
    --------
    dict
        Dictionary containing frequency analysis results
    """
    # Convert window_size from seconds to number of samples
    window_samples = int(window_size * sampling_rate)
    
    # Initialize results dictionary
    results = {}
    
    # Process data in windows
    n_windows = len(data) // window_samples
    all_frequencies = []
    all_power = []
    
    for i in range(n_windows):
        window_data = data['heart_rate'].iloc[i*window_samples:(i+1)*window_samples]
        
        # Calculate PSD using Welch's method
        frequencies, power = signal.welch(
            window_data,
            fs=sampling_rate,
            nperseg=window_samples
        )
        
        all_frequencies.append(frequencies)
        all_power.append(power)
    
    # Average results across windows
    results['frequencies'] = np.mean(all_frequencies, axis=0)
    results['power'] = np.mean(all_power, axis=0)
    
    # Define frequency bands
    bands = {
        'VLF': (0.003, 0.04),
        'LF': (0.04, 0.15),
        'HF': (0.15, 0.4)
    }
    
    # Calculate power in each band
    results['bands'] = {}
    for band_name, (low, high) in bands.items():
        mask = (results['frequencies'] >= low) & (results['frequencies'] <= high)
        results['bands'][band_name] = np.sum(results['power'][mask])
    
    # Calculate LF/HF ratio
    results['bands']['LF/HF'] = results['bands']['LF'] / results['bands']['HF']

    # Save output
    os.makedirs('output', exist_ok = True)
    output_path = os.path.join('output', 'fft_results.npz')
    np.savez(output_path, frequencies = results['frequencies'], power = results['power'], bands = results['bands'])
    
    return results

## 3. Time-Frequency Analysis

Implement the `analyze_time_frequency_features` function to analyze time-frequency features using wavelet transforms.

In [11]:
def analyze_time_frequency_features(data, sampling_rate, window_size=60):
    """Analyze time-frequency features using wavelet transforms.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input data with columns: ['timestamp', 'heart_rate', 'eda', 'temperature', 'subject_id', 'session']
    sampling_rate : float
        Sampling rate of the signal in Hz
    window_size : int, optional
        Size of the analysis window in seconds, default=60
        
    Returns:
    --------
    dict
        Dictionary containing time-frequency analysis results
    """
    # Convert window_size from seconds to number of samples
    window_samples = int(window_size * sampling_rate)
    
    # Initialize results dictionary
    results = {}
    
    # Define wavelet scales
    scales = np.arange(1, 128)
    results['scales'] = scales
    
    # Process data in windows
    n_windows = len(data) // window_samples
    all_coefficients = []
    all_energy = []
    
    for i in range(n_windows):
        window_data = data['heart_rate'].iloc[i*window_samples:(i+1)*window_samples]
        
        # Apply continuous wavelet transform
        coefficients, frequencies = pywt.cwt(
            window_data,
            scales,
            'morl',
            sampling_period=1/sampling_rate
        )
        
        all_coefficients.append(coefficients)
        all_energy.append(np.abs(coefficients)**2)
    
    # Average results across windows
    results['coefficients'] = np.mean(all_coefficients, axis=0)
    results['time_frequency_energy'] = np.mean(all_energy, axis=0)

    # save output
    os.makedirs('output', exist_ok = True)
    output_path = os.path.join('output', 'time_freq_fft_results.npz')
    np.savez(output_path,
             scales = results['scales'],
             coefficients = results['coefficients'],
             energy = results['time_frequency_energy'])
    
    return results

## Example Usage

Here's how to use these functions with your data:

In [13]:
# Load your data
data = pd.read_csv('data/processed/S1.csv')
data_cut = data.head(1000)
data = data_cut
print(data.head())

# Extract time-domain features
features = extract_time_domain_features(data, window_size=60)
print("Time-domain features:")
print(features.head())

# Analyze frequency components
sampling_rate = 4.0  # Hz
freq_results = analyze_frequency_components(data, sampling_rate, window_size=60)
print("\nFrequency analysis results:")
print("Frequency bands:", freq_results['bands'])

# Analyze time-frequency features
tf_results = analyze_time_frequency_features(data, sampling_rate, window_size=60)
print("\nTime-frequency analysis results:")
print("Wavelet scales:", tf_results['scales'].shape)
print("Coefficients shape:", tf_results['coefficients'].shape)

                 timestamp       eda  heart_rate  temperature subject_id  \
0  2018-12-05 16:29:07.000  0.024345  116.000000        21.91         S1   
1  2018-12-05 16:29:07.250  0.023064  107.625004        21.91         S1   
2  2018-12-05 16:29:07.500  0.024345   99.250000        21.91         S1   
3  2018-12-05 16:29:07.750  0.023064   90.874996        21.91         S1   
4  2018-12-05 16:29:08.000  0.023064   82.500000        21.93         S1   

  session  
0   Final  
1   Final  
2   Final  
3   Final  
4   Final  
Time-domain features:
                   timestamp subject_id session  heart_rate_mean  \
240  2018-12-05 16:30:07.000         S1   Final       121.147833   
241  2018-12-05 16:30:07.250         S1   Final       121.246979   
242  2018-12-05 16:30:07.500         S1   Final       121.382271   
243  2018-12-05 16:30:07.750         S1   Final       121.553708   
244  2018-12-05 16:30:08.000         S1   Final       121.761292   

     heart_rate_std  heart_rate_min  hea

NameError: name 'os' is not defined